In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def ideal_low_pass_filter(rows, cols, cutoff):
    """Ideal low-pass filter in the frequency domain."""
    center_row, center_col = rows // 2, cols // 2
    mask = np.zeros((rows, cols, 2), np.float32)
    for i in range(rows):
        for j in range(cols):
            if np.sqrt((i - center_row) ** 2 + (j - center_col) ** 2) <= cutoff:
                mask[i, j] = 1
    return mask

def butterworth_low_pass_filter(rows, cols, cutoff, n):
    """Butterworth low-pass filter in the frequency domain."""
    center_row, center_col = rows // 2, cols // 2
    mask = np.zeros((rows, cols, 2), np.float32)
    for i in range(rows):
        for j in range(cols):
            distance = np.sqrt((i - center_row) ** 2 + (j - center_col) ** 2)
            mask[i, j] = 1 / (1 + (distance / cutoff) ** (2 * n))
    return mask

def gaussian_low_pass_filter(rows, cols, cutoff):
    """Gaussian low-pass filter in the frequency domain."""
    center_row, center_col = rows // 2, cols // 2
    mask = np.zeros((rows, cols, 2), np.float32)
    for i in range(rows):
        for j in range(cols):
            distance = np.sqrt((i - center_row) ** 2 + (j - center_col) ** 2)
            mask[i, j] = np.exp(-(distance ** 2) / (2 * (cutoff ** 2)))
    return mask

def apply_filter(image, mask):
    """Apply the filter to the image in the frequency domain."""
    fft_image = cv2.dft(np.float32(image), flags=cv2.DFT_COMPLEX_OUTPUT)
    fft_shifted = np.fft.fftshift(fft_image)
    filtered = fft_shifted * mask
    filtered_shifted = np.fft.ifftshift(filtered)
    filtered_image = cv2.idft(filtered_shifted)
    filtered_image = cv2.magnitude(filtered_image[:,:,0], filtered_image[:,:,1])
    return cv2.normalize(filtered_image, None, 0, 255, cv2.NORM_MINMAX)

# Load the image
image_path = 'image.jpg' # Replace with the actual path to your image
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Check if the image was loaded successfully
if image is None:
    print(f"Error: Could not load image from {image_path}. Please check the file path.")
else:
    # Compute the size of the image and create masks
    rows, cols = image.shape
    cutoff_frequency = 50  # Adjust this value based on the desired level of smoothing
    n = 2  # For Butterworth filter

    ideal_mask = ideal_low_pass_filter(rows, cols, cutoff_frequency)
    butterworth_mask = butterworth_low_pass_filter(rows, cols, cutoff_frequency, n)
    gaussian_mask = gaussian_low_pass_filter(rows, cols, cutoff_frequency)

    # Apply the filters
    ideal_filtered = apply_filter(image, ideal_mask)
    butterworth_filtered = apply_filter(image, butterworth_mask)
    gaussian_filtered = apply_filter(image, gaussian_mask)

    # Display the images
    plt.figure(figsize=(10, 10))
    plt.subplot(221), plt.imshow(image, cmap='gray'), plt.title('Original Image')
    plt.subplot(222), plt.imshow(ideal_filtered, cmap='gray'), plt.title('Ideal Low-pass Filter')
    plt.subplot(223), plt.imshow(butterworth_filtered, cmap='gray'), plt.title('Butterworth Low-pass Filter')
    plt.subplot(224), plt.imshow(gaussian_filtered, cmap='gray'), plt.title('Gaussian Low-pass Filter')
    plt.show()

Error: Could not load image from image.jpg. Please check the file path.
